# <b>Machine</b> <b>L</b>earning <b>Appl</b>ied to <b>Vio</b>lence <b>De</b>tection in <b>Sur</b>veillance <b>Videos</b> (MachineLApplVioDeSurVideos)

## Development environment setup

In [1]:
%%sx
#@title Colab development environment settings
if [ -n "${COLAB_RELEASE_TAG}" ]; then
    git clone -b main https://github.com/MigeoDaSelva/MachineLApplVioDeSurVideos.git;
    python3 -m pip install --upgrade pip;
    python3 -m pip install -r /content/MachineLApplVioDeSurVideos/requirements.txt;
    kill -9 pid "${PPID}";
fi

[]

In [2]:
#@title Import libraries
from pprint import pprint
import sys
import os

In [3]:
#@title System path manager
%reload_ext autoreload
%autoreload 2

project_name = "MachineLApplVioDeSurVideos"

working_directory = os.popen("echo $PWD").read().rstrip()

project_root_path = working_directory \
    if project_name in working_directory \
        else os.popen(
            f"readlink -f $(find -name {project_name} \
                -not -path '*/drive/*')"
        ).read().rstrip()

if str(project_root_path) not in sys.path:
    sys.path.insert(0, str(project_root_path))

pprint(f"Environment paths: {sys.path}")

("Environment paths: ['/workspaces/MachineLApplVioDeSurVideos', "
 "'/usr/lib/python311.zip', '/usr/lib/python3.11', "
 "'/usr/lib/python3.11/lib-dynload', '', "
 "'/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages']")


In [4]:
#@title Import project stuff
from src.application.evaluation_application import EvaluationApplication
from src.application.execution_application import ExecutionApplication
from src.application.startup_application import StartupApplication
from configs import settings

In [5]:
#@title Colab resource settings
if settings.COLAB_ENV: 
    from google.colab import output
    from google.colab import drive
    output.enable_custom_widget_manager()
    drive.mount("/content/drive", force_remount=True)

In [6]:
#@title Define simulation scope
simulation_scope_settings = StartupApplication.get_scope_manager()

In [7]:
#@title Select data settings
data_settings = StartupApplication.get_data_manager()

IntSlider(value=16, description='Batch size: ', layout=Layout(height='40px', width='auto'), max=1000, min=1, s…

IntSlider(value=60, description='Number of frames: ', layout=Layout(height='40px', width='auto'), max=150, min…

IntSlider(value=224, description='Image height: ', layout=Layout(height='40px', width='auto'), max=2000, min=1…

IntSlider(value=224, description='Image width: ', layout=Layout(height='40px', width='auto'), max=2000, min=1,…

IntSlider(value=3, description='Number of channels: ', layout=Layout(height='40px', width='auto'), max=10, min…

In [8]:
#@title Select approach settings
approach_settings = StartupApplication.get_approach_manager()

Dropdown(description='Approach options: ', layout=Layout(height='40px', width='auto'), options=(('(2+1)D-CNN',…

Dropdown(description='Optimizer options: ', layout=Layout(height='40px', width='auto'), options=(('SGD', <clas…

Dropdown(description='Activation options: ', layout=Layout(height='40px', width='auto'), options=('relu', 'sig…

Dropdown(description='Padding options: ', layout=Layout(height='40px', width='auto'), options=('same', 'valid'…

IntSlider(value=300, description='Epochs: ', layout=Layout(height='40px', width='auto'), max=300, min=1, style…

FloatLogSlider(value=32.0, base=2.0, description='Filters: ', layout=Layout(height='40px', width='auto'), max=…

FloatSlider(value=0.1, continuous_update=False, description='Dropout: ', layout=Layout(height='40px', width='a…

FloatLogSlider(value=0.01, description='Learning rate: ', layout=Layout(height='40px', width='auto'), max=-1.0…

Accordion(children=(Tab(children=(VBox(children=(Checkbox(value=False, description='Disabled', indent=False),)…

In [9]:
#@title Show selected values

print(
    "\nSimulation scope sttings:\n",
    f"Iterations: {simulation_scope_settings.iterations}",
    f"folds: {simulation_scope_settings.folds}",
    sep="\n"
)

print(
    "\nSelected data attributes:\n",
    f"Number of channels: {data_settings.n_channels}",
    f"Number of frames: {data_settings.n_frames}",
    f"Image height: {data_settings.image_height}",
    f"Image width: {data_settings.image_width}",
    f"Batch size: {data_settings.batch_size}",
    sep = "\n"
)

print(
    f"\nSelected {approach_settings.approach.__name__} attributes:\n",
    f"Callbacks: {[callback.__class__.__name__ for callback in approach_settings.callbacks]}",
    f"Number of output initial filters: {approach_settings.filters}",
    f"Pooling Strides: {approach_settings.pooling_strides}",
    f"Activation function: {approach_settings.activation}",
    f"Load latest model: {approach_settings.continue_training}",
    f"Padding approach: {approach_settings.padding}",
    f"Kernel size: {approach_settings.kernel_size}",
    f"Number of epochs: {approach_settings.epochs}",
    f"Learning rate: {approach_settings.learning_rate}",
    f"Unfreezing: {approach_settings.unfreezing}",
    f"Optimizer: {approach_settings.optimizer.__name__}",
    f"Dropout: {approach_settings.dropout}",
    sep = "\n"
)


Simulation scope sttings:

Iterations: [0]
folds: [0]

Selected data attributes:

Number of channels: 3
Number of frames: 1
Image height: 224
Image width: 224
Batch size: 1

Selected Movinet attributes:

Callbacks: ['ModelCheckpoint', 'CSVLogger', 'EarlyStopping', 'LearningRateScheduler']
Number of output initial filters: 32
Pooling Strides: (2, 2, 2)
Activation function: relu
Load latest model: True
Padding approach: same
Kernel size: (3, 3, 3)
Number of epochs: 1
Learning rate: 0.01
Unfreezing: False
Optimizer: SGD
Dropout: 0.1


In [10]:
#@title build pipeline
pipeline = ExecutionApplication.get_pipeline(
    simulation_scope_settings=simulation_scope_settings,
    data_settings=data_settings,
    approach_settings=approach_settings
)

In [16]:
#@title Pipeline runimg
pipeline.run()

AttributeError: 'MovinetClassifier' object has no attribute 'layer'

In [ ]:
#@title Load latest best model
ExecutionApplication.load_best_model(pipeline.approach)

In [ ]:
#@title Evaluate
evaluation_application = EvaluationApplication(
    approach=pipeline.approach,
    data_settings=data_settings,
    simulation_scope_settings=simulation_scope_settings,
)

evaluation_application.evaluate()
evaluation_application.calculate_classification_metrics()
evaluation_application.save_confusion_matrix()

In [ ]:
#@title Post-training
evaluation_application.calculate_average_classification_metrics()
evaluation_application.build_confusion_matrix()